In [1]:
import numpy as np #vector operations
import pandas as pd #label data
import matplotlib.pyplot as plt #visualize data

import pickle #to use save and reuse model
import itertools
from mpl_toolkits.mplot3d import Axes3D
import re

In [12]:
#loading data
with open('raw_data.pkl', 'rb') as handle:
    raw_data = pickle.load(handle)

In [13]:
filtered_data = []
listTrim = []
error = []
#things that we want in DF
attributes = ["Trim", "Year", "Colour", "Drivetrain", "Kilometers"]

#loop through attripbutes
for k in range(len(attributes)):
    #list of that specfic attribute
    list = []
    errorCache = []
    #loop through data
    for i in range(len(raw_data)):
        #length of description
        lenAttr = len(raw_data[i][3][0])
        appended = False
        #loop thru description
        for j in range(lenAttr):
            #if description is what we are looking for
            if raw_data[i][3][0][j] == attributes[k]:
                #list append data
                list.append(raw_data[i][3][1][j])
                appended = True
        if appended != True:
            errorCache.append(i)
            list.append("NaN")
    filtered_data.append(list)
    error.append(errorCache)

In [14]:
def convertStrToNum(str):
    num = "".join([s for s in str if s.isdigit()])
    if len(num) > 0:
        return int(num)
    else:
        return np.nan

In [15]:
def freqCount(list):
    return {i: list.count(i) for i in list}

Data Gathering

In [16]:
# price
Y = np.zeros((len(raw_data),1))
for i in range(len(raw_data)):
    Y[i,0] = convertStrToNum(raw_data[i][0])

In [17]:
kilometers = np.zeros((len(raw_data),1))
for i in range(len(raw_data)):
    kilometers[i,0] = convertStrToNum(filtered_data[-1][i])

In [18]:
m = len(filtered_data[-2])
drivetrain = np.zeros((len(raw_data),1))
for i in range(m):
    dt = filtered_data[-2][i]
    if dt == "All-wheel drive (AWD)" or dt == "4 x 4":
        drivetrain[i,0] = 1
    elif dt == 'Front-wheel drive (FWD)' or dt == 'Rear-wheel drive (RWD)':
        drivetrain[i,0] = 0
    else:
        drivetrain[i,0] = np.nan
        

In [30]:
colour_index = {'Black': 0,
 'Blue': 1,
 'Brown': 2,
 'Burgundy': 3,
 'Gold': 4,
 'Green': 5,
 'Grey': 6,
 'Purple': 7,
 'Red': 8,
 'Silver': 9,
 'Tan': 10,
 'White': 11}

colours = np.zeros((m,1))

for i in range(m):
    colour = filtered_data[2][i]
    if colour == "NaN" or colour == "Other":
        colours[i,0] = np.NaN
    else:
        colours[i,0] = colourIndex[colour]

'Black'

In [31]:
filtered_data[1]
year = np.zeros((m,1))
for i in range(m):
    year[i,0] = 2018 - convertStrToNum(filtered_data[1][i])

In [45]:
def findAttributes(tags, data, setNull=False):
    npData = np.zeros((m,1))
    
    for i in range(len(data)):
        lineData = data[i]
        containTag = False
        for j in range(len(lineData)):
            for tag in tags:
                value = lineData[j:j+len(tag)].lower()
                if value == tag:
                    npData[i,0] = tags[value]
                    containTag = True
        if not containTag and setNull:
            npData[i,0] = np.nan
    return npData

In [46]:
leather_tags = {
    "leather": 1,
    "lthr": 1,
    "leathr": 1,
    "letr":1,
    "leath":1
}
trim_tags = {
    "ls": 0,
    "lt": 1,
    "lt2": 2,
    "2lt": 2,
    "ltz": 3
}
camera_tags = {
    "cam": 1,
    "rear": 1,
    "camera": 1,
    "cmr":1,
    "mera":1,
}
blue_tags = {
    "bluetooth": 1,
    "tooth": 1,
    "toth": 1,
    "blue":1,
    "bth":1,
    "btoth":1
}
cruise_tags = {
    "cc ": 1,
    "cc/": 1,
    "cc,": 1,
    "cc.": 1,
    "cc'": 1,
    "cc;": 1,
    "cruise control": 1,
    "cruise":1,
    "crus": 1,
    "cru": 1
}
remote_tags = {
    "remote control": 1,
    "remote": 1,
    "rcontrol": 1,
    "rmt": 1,
}
sunroof_tags = {
    "sunroof": 1,
    "sun roof": 1,
    "sun_roof": 1,
    "roof": 1,
    "sun": 1,
}
heated_tags = {
    "heated": 1,
    "heated seat": 1,
    "heatedseats": 1,
    "hseats": 1,
    "heat": 1,
}
leatherTest = findAttributes(leather_tags, filtered_data[0])
trimTest = findAttributes(trim_tags, filtered_data[0], setNull=True)
cameraTest = findAttributes(camera_tags, filtered_data[0])
blueTest = findAttributes(blue_tags, filtered_data[0])
remoteTest = findAttributes(remote_tags, filtered_data[0])
sunroofTest = findAttributes(sunroof_tags, filtered_data[0])
heatedTest = findAttributes(heated_tags, filtered_data[0])


print("leather", end="")
print(np.unique(leatherTest[~np.isnan(leatherTest)], return_counts=True))
print("trim", end="")
print(np.unique(trimTest[~np.isnan(trimTest)], return_counts=True))
print("camera", end="")
print(np.unique(cameraTest[~np.isnan(cameraTest)], return_counts=True))
print("blue", end="")
print(np.unique(blueTest[~np.isnan(blueTest)], return_counts=True))
print("remote", end="")
print(np.unique(remoteTest[~np.isnan(remoteTest)], return_counts=True))
print("sunroof", end="")
print(np.unique(sunroofTest[~np.isnan(sunroofTest)], return_counts=True))
print("heated", end="")
print(np.unique(heatedTest[~np.isnan(heatedTest)], return_counts=True))

leather(array([ 0.,  1.]), array([2371,   98]))
trim(array([ 0.,  1.,  2.,  3.]), array([ 728, 1153,   16,  158]))
camera(array([ 0.,  1.]), array([2311,  158]))
blue(array([ 0.,  1.]), array([2320,  149]))
remote(array([ 0.,  1.]), array([2392,   77]))
sunroof(array([ 0.,  1.]), array([2355,  114]))
heated(array([ 0.,  1.]), array([2362,  107]))


In [52]:
X = np.column_stack((kilometers, drivetrain, colours, year, trimTest, leatherTest, cameraTest, blueTest, remoteTest, sunroofTest, heatedTest, Y))

In [49]:
X

array([[  1.63656000e+05,   3.19000000e+04,   1.30000000e+05, ...,
          3.10810000e+04,   1.28000000e+05,   1.23770000e+04],
       [  1.00000000e+00,   1.00000000e+00,   0.00000000e+00, ...,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   1.10000000e+01,   0.00000000e+00, ...,
          0.00000000e+00,   1.10000000e+01,   0.00000000e+00],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  8.56200000e+05,   1.79950000e+06,   9.19700000e+05, ...,
          2.39880000e+06,   1.14950000e+06,   3.48260000e+06]])

In [53]:
with open("comprised_data.pkl", "wb") as output_file:
    pickle.dump(X, output_file)